# **Chapter 21: GraphQL & WebSockets**

---

## **Learning Objectives**

By the end of this chapter, you will be able to:

- Set up and configure GraphQL clients in Flutter for efficient data fetching
- Execute queries, mutations, and subscriptions with proper error handling
- Establish WebSocket connections for real-time bidirectional communication
- Implement Socket.IO for event-driven real-time features
- Handle connection state management, reconnection logic, and heartbeat mechanisms
- Implement Server-Sent Events (SSE) for server-to-client push notifications
- Optimize network usage with query batching, caching strategies, and persisted queries

---

## **Prerequisites**

- Completed Chapter 19: HTTP Requests & REST APIs
- Completed Chapter 20: Authentication & Security (JWT handling)
- Understanding of State Management (Chapter 12-15) for managing connection states
- Basic understanding of GraphQL schema concepts (types, resolvers)
- Familiarity with Stream controllers from Chapter 6 (Asynchronous Programming)

---

## **21.1 GraphQL Client Setup**

GraphQL is a query language for APIs that allows clients to request exactly the data they need, reducing over-fetching and under-fetching common in REST APIs.

### **GraphQL Client Configuration**

```dart
// graphql_config.dart
// Configuration for GraphQL client with authentication and caching

import 'package:flutter/foundation.dart';
import 'package:graphql_flutter/graphql_flutter.dart';
import 'package:http/http.dart' as http;
import 'package:http/io_client.dart';
import '../services/secure_storage_service.dart';

/// Configures and provides GraphQL client instances
/// Handles authentication, caching, and error link configuration
class GraphQLConfig {
  // Secure storage for retrieving auth tokens
  final SecureStorageService _secureStorage = SecureStorageService();
  
  // GraphQL endpoint configuration
  static const String httpEndpoint = 'https://api.example.com/graphql';
  static const String wsEndpoint = 'wss://api.example.com/graphql';
  
  /// Creates the main GraphQL client with full configuration
  /// Includes auth link, error link, HTTP link, and cache policy
  Future<GraphQLClient> getClient() async {
    // Initialize Hive store for persistent cache (offline support)
    // Hive is a lightweight key-value database written in Dart
    final store = await HiveStore.open();
    
    // GraphQL cache configuration
    // Optimistic results and normalization for efficient updates
    final cache = GraphQLCache(
      store: store,
      dataIdFromObject: _dataIdFromObject,  // Custom ID normalization
    );
    
    // Link chain: Auth -> Error -> HTTP
    // Links are composed from right to left (HTTP is the terminating link)
    final Link link = Link.split(
      (request) => request.isSubscription,  // Check if request is subscription
      _getWebSocketLink(),                   // Use WebSocket for subscriptions
      _getAuthLink().concat(_getErrorLink().concat(_getHttpLink())),  // HTTP for queries/mutations
    );
    
    return GraphQLClient(
      link: link,
      cache: cache,
      defaultPolicies: DefaultPolicies(
        query: Policies(
          fetch: FetchPolicy.cacheAndNetwork,  // Show cached data, then refresh
          error: ErrorPolicy.all,              // Return partial data if available
        ),
        mutate: Policies(
          fetch: FetchPolicy.noCache,          // Don't cache mutations
        ),
      ),
    );
  }
  
  /// Authentication link: Adds JWT token to request headers
  Link _getAuthLink() {
    return AuthLink(
      getToken: () async {
        // Retrieve token from secure storage
        final token = await _secureStorage.getAccessToken();
        
        if (token == null) return null;
        
        // Check if token is expired before using
        final isExpired = await _secureStorage.isTokenExpired();
        if (isExpired) {
          // Attempt refresh or return null to trigger re-auth
          final refreshed = await _attemptTokenRefresh();
          if (!refreshed) return null;
          
          // Get new token after refresh
          return 'Bearer ${await _secureStorage.getAccessToken()}';
        }
        
        return 'Bearer $token';
      },
      headerKey: 'Authorization',  // Header name: Authorization: Bearer <token>
    );
  }
  
  /// Error handling link: Centralized error processing
  Link _getErrorLink() {
    return ErrorLink(
      onException: (request, forward, exception) {
        // Handle network exceptions (no connection, timeout)
        debugPrint('GraphQL Network Exception: ${exception.toString()}');
        
        // You can transform exceptions into GraphQL errors here
        // or implement retry logic
      },
      onGraphQLError: (request, forward, response) {
        // Handle GraphQL-specific errors (validation errors, resolver errors)
        final errors = response.errors;
        
        if (errors != null) {
          for (final error in errors) {
            debugPrint('GraphQL Error: ${error.message}');
            
            // Check for authentication errors
            if (error.extensions?['code'] == 'UNAUTHENTICATED') {
              // Trigger logout or token refresh
              _handleAuthError();
            }
          }
        }
        
        // Return response to allow partial data handling
        return response;
      },
    );
  }
  
  /// HTTP link: Handles actual HTTP transport
  Link _getHttpLink() {
    return HttpLink(
      httpEndpoint,
      httpClient: http.Client(),  // Can use custom HTTP client with pinning
      defaultHeaders: {
        'Content-Type': 'application/json',
        'Accept': 'application/json',
      },
    );
  }
  
  /// WebSocket link: For subscriptions (real-time updates)
  Link _getWebSocketLink() {
    return WebSocketLink(
      wsEndpoint,
      config: SocketClientConfig(
        autoReconnect: true,           // Automatically reconnect on disconnect
        inactivityTimeout: Duration(seconds: 30),  // Keep-alive timeout
        initialPayload: () async {
          // Send auth token on connection init
          final token = await _secureStorage.getAccessToken();
          return {
            'Authorization': 'Bearer $token',
          };
        },
      ),
    );
  }
  
  /// Custom function to generate cache IDs from objects
  /// Ensures consistent cache normalization across types
  String? _dataIdFromObject(Object object) {
    // Handle Flutter type casting
    if (object is Map<String, dynamic>) {
      // If object has id field, use TypeName:id as cache key
      // This is the standard GraphQL normalization strategy
      if (object.containsKey('id') && object.containsKey('__typename')) {
        return '${object['__typename']}:${object['id']}';
      }
    }
    return null;  // Let default handler take over
  }
  
  /// Attempts to refresh expired token
  Future<bool> _attemptTokenRefresh() async {
    try {
      final refreshToken = await _secureStorage.getRefreshToken();
      if (refreshToken == null) return false;
      
      // Implement refresh logic here
      // final newTokens = await authService.refreshToken(refreshToken);
      // await _secureStorage.storeAuthTokens(...);
      
      return true;
    } catch (e) {
      return false;
    }
  }
  
  /// Handles authentication errors globally
  void _handleAuthError() {
    // Navigate to login or show auth expired dialog
    // This would typically use a navigator key or event bus
    debugPrint('Authentication error: Token invalid or expired');
  }
}

/// Provider for GraphQL client to use with InheritedWidget or Provider
class GraphQLProviderWidget extends StatelessWidget {
  final Widget child;
  
  const GraphQLProviderWidget({Key? key, required this.child}) : super(key: key);
  
  @override
  Widget build(BuildContext context) {
    return FutureBuilder<GraphQLClient>(
      future: GraphQLConfig().getClient(),
      builder: (context, snapshot) {
        if (snapshot.connectionState == ConnectionState.waiting) {
          return Material(
            child: Center(child: CircularProgressIndicator()),
          );
        }
        
        if (snapshot.hasError) {
          return Material(
            child: Center(child: Text('Failed to initialize GraphQL: ${snapshot.error}')),
          );
        }
        
        // Provide GraphQL client to widget tree
        return GraphQLProvider(
          client: ValueNotifier(snapshot.data!),
          child: CacheProvider(child: child),
        );
      },
    );
  }
}
```

**Explanation:**

- **`HiveStore`**: A persistent cache implementation using Hive (NoSQL database). Unlike in-memory cache, this survives app restarts and enables offline-first capabilities.
- **`GraphQLCache`**: Normalizes GraphQL responses by merging objects with the same `__typename` and `id`, preventing data duplication and ensuring UI consistency when the same object appears in different queries.
- **Link Composition**: Links are middleware that process requests. They chain from left to right: `AuthLink` adds headers, `ErrorLink` processes errors, `HttpLink` sends the HTTP request.
- **`Link.split`**: Routes requests based on a condition. Subscriptions go to WebSocket, queries/mutations go to HTTP. This is efficient because subscriptions need persistent connections while queries use request-response.
- **`FetchPolicy.cacheAndNetwork`**: Returns cached data immediately for fast UI rendering, then fetches fresh data from network to update the UI. This gives the perception of instant loading while ensuring data freshness.
- **`initialPayload`**: WebSocket connections require authentication on connection establishment (unlike HTTP where each request has headers). This sends the JWT token in the connection initialization message.

### **GraphQL Queries and Mutations**

```dart
// graphql_operations.dart
// Example GraphQL operations with Flutter widgets

import 'package:flutter/material.dart';
import 'package:graphql_flutter/graphql_flutter.dart';

/// Query example: Fetching user profile with variables
class UserProfileQuery extends StatelessWidget {
  final String userId;
  
  const UserProfileQuery({Key? key, required this.userId}) : super(key: key);
  
  // GraphQL query with variable interpolation
  // $userId is a variable of type ID! (non-null ID)
  static const String fetchUserQuery = r'''
    query GetUser($userId: ID!) {
      user(id: $userId) {
        id
        name
        email
        avatar
        posts(limit: 10) {
          id
          title
          content
          createdAt
        }
      }
    }
  ''';
  
  @override
  Widget build(BuildContext context) {
    return Query(
      options: QueryOptions(
        document: gql(fetchUserQuery),  // Parse string to AST
        variables: {'userId': userId},   // Pass variables
        fetchPolicy: FetchPolicy.cacheFirst,  // Try cache, then network
        pollInterval: Duration(minutes: 5),   // Refetch every 5 minutes
      ),
      builder: (QueryResult result, {VoidCallback? refetch, FetchMore? fetchMore}) {
        // Handle loading state
        if (result.isLoading && result.data == null) {
          return Center(child: CircularProgressIndicator());
        }
        
        // Handle errors
        if (result.hasException) {
          return ErrorWidget(
            error: result.exception!,
            onRetry: refetch,  // Pass refetch callback for retry button
          );
        }
        
        // Extract data (nullable because GraphQL returns null for missing fields)
        final userData = result.data?['user'];
        if (userData == null) {
          return Center(child: Text('User not found'));
        }
        
        // Render UI with data
        return UserProfileView(
          user: User.fromJson(userData),
          onRefresh: () async {
            // Refetch returns Future, await for refresh indicator
            await refetch?.call();
          },
          onLoadMore: () {
            // Pagination with fetchMore
            fetchMore?.call(
              FetchMoreOptions(
                variables: {'limit': 10, 'offset': 10},
                updateQuery: (existing, fresh) {
                  // Merge existing posts with new posts
                  final List<dynamic> posts = [
                    ...existing?['user']['posts'] ?? [],
                    ...fresh?['user']['posts'] ?? [],
                  ];
                  
                  return {
                    'user': {
                      ...existing?['user'],
                      'posts': posts,
                    }
                  };
                },
              ),
            );
          },
        );
      },
    );
  }
}

/// Mutation example: Creating a new post
class CreatePostMutation extends StatefulWidget {
  @override
  _CreatePostMutationState createState() => _CreatePostMutationState();
}

class _CreatePostMutationState extends State<CreatePostMutation> {
  final TextEditingController _titleController = TextEditingController();
  final TextEditingController _contentController = TextEditingController();
  
  // Mutation document with input type
  static const String createPostMutation = r'''
    mutation CreatePost($input: CreatePostInput!) {
      createPost(input: $input) {
        id
        title
        content
        createdAt
        author {
          id
          name
        }
      }
    }
  ''';
  
  @override
  Widget build(BuildContext context) {
    return Mutation(
      options: MutationOptions(
        document: gql(createPostMutation),
        // Optimistic update: Update UI before server responds
        optimisticResult: {
          'createPost': {
            'id': 'temp-${DateTime.now().millisecondsSinceEpoch}',
            'title': _titleController.text,
            'content': _contentController.text,
            'createdAt': DateTime.now().toIso8601String(),
            'author': {
              'id': 'current-user-id',
              'name': 'Current User',
            },
          },
        },
        // Update cache after mutation succeeds
        update: (cache, result) {
          if (result?.data == null) return;
          
          // Read existing posts query from cache
          final existingPosts = cache.readQuery(
            Request(
              operation: Operation(
                document: gql(UserProfileQuery.fetchUserQuery),
              ),
            ),
          );
          
          if (existingPosts != null) {
            // Add new post to beginning of list
            final newPosts = [
              result!.data!['createPost'],
              ...existingPosts['user']['posts'],
            ];
            
            // Write updated data back to cache
            cache.writeQuery(
              Request(
                operation: Operation(
                  document: gql(UserProfileQuery.fetchUserQuery),
                ),
              ),
              data: {
                'user': {
                  ...existingPosts['user'],
                  'posts': newPosts,
                },
              },
            );
          }
        },
        // Handle errors
        onError: (error) {
          ScaffoldMessenger.of(context).showSnackBar(
            SnackBar(content: Text('Failed to create post: $error')),
          );
        },
        onCompleted: (data) {
          // Clear form on success
          _titleController.clear();
          _contentController.clear();
          Navigator.pop(context);
        },
      ),
      builder: (RunMutation runMutation, QueryResult? result) {
        return Padding(
          padding: EdgeInsets.all(16),
          child: Column(
            children: [
              TextField(
                controller: _titleController,
                decoration: InputDecoration(labelText: 'Title'),
              ),
              TextField(
                controller: _contentController,
                decoration: InputDecoration(labelText: 'Content'),
                maxLines: 5,
              ),
              SizedBox(height: 20),
              ElevatedButton(
                onPressed: result?.isLoading ?? false
                    ? null  // Disable while loading
                    : () => runMutation({
                          'input': {
                            'title': _titleController.text,
                            'content': _contentController.text,
                          },
                        }),
                child: result?.isLoading ?? false
                    ? CircularProgressIndicator()
                    : Text('Create Post'),
              ),
            ],
          ),
        );
      },
    );
  }
}

/// Error widget for GraphQL errors
class ErrorWidget extends StatelessWidget {
  final OperationException error;
  final VoidCallback? onRetry;
  
  const ErrorWidget({Key? key, required this.error, this.onRetry}) : super(key: key);
  
  @override
  Widget build(BuildContext context) {
    // Extract user-friendly error messages
    final messages = error.graphqlErrors
        .map((e) => e.message)
        .toList();
    
    if (error.linkException != null) {
      messages.add('Network error. Please check your connection.');
    }
    
    return Center(
      child: Column(
        mainAxisAlignment: MainAxisAlignment.center,
        children: [
          Icon(Icons.error_outline, color: Colors.red, size: 48),
          SizedBox(height: 16),
          Text(
            'Something went wrong',
            style: Theme.of(context).textTheme.headlineSmall,
          ),
          SizedBox(height: 8),
          ...messages.map((m) => Padding(
                padding: EdgeInsets.symmetric(horizontal: 32),
                child: Text(m, textAlign: TextAlign.center),
              )),
          SizedBox(height: 16),
          if (onRetry != null)
            ElevatedButton.icon(
              onPressed: onRetry,
              icon: Icon(Icons.refresh),
              label: Text('Retry'),
            ),
        ],
      ),
    );
  }
}
```

**Explanation:**

- **`gql()`**: Parses the GraphQL query string into an Abstract Syntax Tree (AST). This is done at runtime but can be optimized with code generation (graphql_codegen) to parse at build time for better performance.
- **`variables`**: GraphQL variables are type-safe replacements for string interpolation. They prevent injection attacks and allow query reuse with different parameters.
- **`pollInterval`**: Automatically re-executes the query at the specified interval. Useful for data that changes frequently but doesn't need real-time updates (like dashboards).
- **`optimisticResult`**: Provides fake data to the UI immediately while the mutation runs. When the server responds, the real data replaces the optimistic data. This makes the app feel instant.
- **`update` callback**: Manually updates the cache after a mutation. Without this, the new post wouldn't appear in the list until the next query refetch. The cache update merges the new post into the existing `user.posts` array.
- **`FetchMore`**: Implements cursor-based or offset-based pagination. The `updateQuery` function merges the new page of results with existing data in the cache.

---

## **21.2 GraphQL Subscriptions**

Subscriptions maintain a persistent WebSocket connection to receive real-time updates from the server when data changes.

### **Implementing Real-time Subscriptions**

```dart
// subscriptions.dart
// Real-time GraphQL subscriptions for live updates

import 'package:flutter/material.dart';
import 'package:graphql_flutter/graphql_flutter.dart';

/// Subscription for real-time notifications
/// Listens to new notification events for the current user
class NotificationSubscription extends StatefulWidget {
  @override
  _NotificationSubscriptionState createState() => _NotificationSubscriptionState();
}

class _NotificationSubscriptionState extends State<NotificationSubscription> {
  // Subscription document
  // Subscriptions use the 'subscription' keyword instead of 'query'
  static const String notificationSub = r'''
    subscription OnNewNotification {
      notificationReceived {
        id
        type
        message
        timestamp
        read
        metadata {
          actionUrl
          imageUrl
        }
      }
    }
  ''';
  
  @override
  Widget build(BuildContext context) {
    return Subscription(
      options: SubscriptionOptions(
        document: gql(notificationSub),
        // Reconnect if connection drops
        reconnectOnConnectionLost: true,
      ),
      builder: (result) {
        // Handle connection states
        if (result.isLoading) {
          return ConnectionStatusIndicator(
            status: ConnectionStatus.connecting,
          );
        }
        
        if (result.hasException) {
          return ConnectionStatusIndicator(
            status: ConnectionStatus.error,
            error: result.exception.toString(),
          );
        }
        
        // Handle incoming data
        if (result.data != null) {
          final notification = result.data!['notificationReceived'];
          
          // Show in-app notification or update badge
          WidgetsBinding.instance.addPostFrameCallback((_) {
            _showInAppNotification(context, notification);
          });
        }
        
        // Subscription doesn't render UI directly, just listens
        return SizedBox.shrink();
      },
    );
  }
  
  void _showInAppNotification(BuildContext context, Map<String, dynamic> notification) {
    final messenger = ScaffoldMessenger.of(context);
    messenger.showSnackBar(
      SnackBar(
        content: ListTile(
          leading: notification['metadata']?['imageUrl'] != null
              ? CircleAvatar(
                  backgroundImage: NetworkImage(notification['metadata']['imageUrl']),
                )
              : Icon(Icons.notifications),
          title: Text(notification['type']),
          subtitle: Text(notification['message']),
        ),
        action: SnackBarAction(
          label: 'View',
          onPressed: () {
            // Navigate to notification detail
            final url = notification['metadata']?['actionUrl'];
            if (url != null) {
              Navigator.pushNamed(context, '/notifications');
            }
          },
        ),
        duration: Duration(seconds: 5),
        behavior: SnackBarBehavior.floating,
      ),
    );
  }
}

/// Real-time chat subscription
/// Demonstrates parameterized subscriptions
class ChatMessageSubscription extends StatelessWidget {
  final String chatRoomId;
  
  const ChatMessageSubscription({Key? key, required this.chatRoomId}) : super(key: key);
  
  static const String messageSub = r'''
    subscription OnNewMessage($chatRoomId: ID!) {
      messageReceived(chatRoomId: $chatRoomId) {
        id
        content
        sender {
          id
          name
          avatar
        }
        timestamp
        attachments {
          type
          url
        }
      }
    }
  ''';
  
  @override
  Widget build(BuildContext context) {
    return Subscription(
      options: SubscriptionOptions(
        document: gql(messageSub),
        variables: {'chatRoomId': chatRoomId},
      ),
      builder: (result) {
        // Connection state handling is crucial for UX
        if (!result.isLoading && !result.hasException && result.data == null) {
          // Connected but no data yet (waiting for events)
          return ConnectionStatusIndicator(status: ConnectionStatus.connected);
        }
        
        if (result.hasException) {
          return Column(
            children: [
              ConnectionStatusIndicator(
                status: ConnectionStatus.error,
                error: result.exception?.graphqlErrors.first.message,
              ),
              ElevatedButton(
                onPressed: () {
                  // Trigger manual reconnect
                  // This requires access to client, typically via context
                },
                child: Text('Reconnect'),
              ),
            ],
          );
        }
        
        // Build message list with real-time updates
        return MessageList(
          newMessage: result.data?['messageReceived'],
          chatRoomId: chatRoomId,
        );
      },
    );
  }
}

/// Connection status indicator widget
enum ConnectionStatus { connecting, connected, error }

class ConnectionStatusIndicator extends StatelessWidget {
  final ConnectionStatus status;
  final String? error;
  
  const ConnectionStatusIndicator({
    Key? key,
    required this.status,
    this.error,
  }) : super(key: key);
  
  @override
  Widget build(BuildContext context) {
    Color color;
    IconData icon;
    String message;
    
    switch (status) {
      case ConnectionStatus.connecting:
        color = Colors.orange;
        icon = Icons.sync;
        message = 'Connecting...';
        break;
      case ConnectionStatus.connected:
        color = Colors.green;
        icon = Icons.check_circle;
        message = 'Live';
        break;
      case ConnectionStatus.error:
        color = Colors.red;
        icon = Icons.error;
        message = error ?? 'Connection lost';
        break;
    }
    
    return Container(
      padding: EdgeInsets.symmetric(horizontal: 12, vertical: 6),
      decoration: BoxDecoration(
        color: color.withOpacity(0.1),
        borderRadius: BorderRadius.circular(16),
        border: Border.all(color: color),
      ),
      child: Row(
        mainAxisSize: MainAxisSize.min,
        children: [
          Icon(icon, color: color, size: 16),
          SizedBox(width: 6),
          Text(
            message,
            style: TextStyle(color: color, fontSize: 12),
          ),
        ],
      ),
    );
  }
}

/// Managing multiple subscriptions efficiently
/// Uses StreamBuilder pattern for complex real-time UIs
class MultiSubscriptionManager extends StatefulWidget {
  @override
  _MultiSubscriptionManagerState createState() => _MultiSubscriptionManagerState();
}

class _MultiSubscriptionManagerState extends State<MultiSubscriptionManager> {
  // Multiple subscription streams
  Stream<QueryResult>? _notificationStream;
  Stream<QueryResult>? _presenceStream;
  
  @override
  void didChangeDependencies() {
    super.didChangeDependencies();
    
    // Get GraphQL client from context
    final client = GraphQLProvider.of(context).value;
    
    // Create subscription operations
    final notificationRequest = Operation(
      document: gql(r'''
        subscription { notificationReceived { id message } }
      '''),
    );
    
    final presenceRequest = Operation(
      document: gql(r'''
        subscription { userPresenceUpdated { userId status } }
      '''),
    );
    
    // Subscribe to multiple streams
    _notificationStream = client.subscribe(notificationRequest);
    _presenceStream = client.subscribe(presenceRequest);
  }
  
  @override
  Widget build(BuildContext context) {
    return StreamBuilder<QueryResult>(
      stream: _notificationStream,
      builder: (context, notificationSnapshot) {
        return StreamBuilder<QueryResult>(
          stream: _presenceStream,
          builder: (context, presenceSnapshot) {
            // Combine data from both subscriptions
            final hasNotification = notificationSnapshot.hasData;
            final onlineUsers = presenceSnapshot.data?.data?['userPresenceUpdated'] ?? [];
            
            return Column(
              children: [
                if (hasNotification)
                  Banner(
                    message: 'New!',
                    location: BannerLocation.topEnd,
                    child: Icon(Icons.notifications),
                  ),
                Text('$onlineUsers users online'),
              ],
            );
          },
        );
      },
    );
  }
}
```

**Explanation:**

- **`subscription` keyword**: GraphQL subscriptions use the `subscription` operation type instead of `query` or `mutation`. The server pushes data to the client when the subscribed event occurs.
- **WebSocket Transport**: Subscriptions require a WebSocket connection (`WebSocketLink`) instead of HTTP because they need a persistent, bidirectional channel.
- **Connection States**: Subscriptions go through states: `connecting` (WebSocket handshake), `connected` (listening), and potential `error` (network issues or auth failures).
- **Automatic Reconnection**: The `reconnectOnConnectionLost` option ensures the client attempts to re-establish the WebSocket connection if it drops (e.g., when switching from WiFi to cellular).
- **Multiple Subscriptions**: You can subscribe to multiple events simultaneously. Each `Subscription` widget maintains its own WebSocket frame listener.
- **Parameterized Subscriptions**: Like queries, subscriptions accept variables. This allows subscribing to specific chat rooms, user IDs, or topics rather than broadcast channels.

---

## **21.3 WebSocket Connections**

For non-GraphQL real-time communication, use direct WebSocket connections for chat, gaming, or live data streaming.

### **WebSocket Client Implementation**

```dart
// websocket_service.dart
// Generic WebSocket service with reconnection and heartbeat

import 'dart:async';
import 'dart:convert';
import 'dart:io';
import 'package:flutter/foundation.dart';
import 'package:web_socket_channel/web_socket_channel.dart';
import 'package:web_socket_channel/io.dart';

/// Generic WebSocket service with enterprise features:
/// - Automatic reconnection with exponential backoff
/// - Heartbeat/ping-pong to detect dead connections
/// - Message queuing while offline
/// - Connection state management
class WebSocketService {
  // Connection configuration
  final String wsUrl;
  final Duration reconnectInterval;
  final Duration heartbeatInterval;
  final int maxReconnectAttempts;
  
  WebSocketChannel? _channel;
  StreamSubscription? _subscription;
  Timer? _heartbeatTimer;
  Timer? _reconnectTimer;
  
  // State management
  ConnectionState _state = ConnectionState.disconnected;
  int _reconnectAttempts = 0;
  bool _shouldReconnect = true;
  bool _isDisposed = false;
  
  // Message queue for offline buffering
  final List<String> _messageQueue = [];
  
  // Stream controllers for public API
  final _messageController = StreamController<dynamic>.broadcast();
  final _stateController = StreamController<ConnectionState>.broadcast();
  
  // Getters for streams
  Stream<dynamic> get messageStream => _messageController.stream;
  Stream<ConnectionState> get stateStream => _stateController.stream;
  ConnectionState get currentState => _state;
  bool get isConnected => _state == ConnectionState.connected;
  
  WebSocketService({
    required this.wsUrl,
    this.reconnectInterval = const Duration(seconds: 5),
    this.heartbeatInterval = const Duration(seconds: 30),
    this.maxReconnectAttempts = 10,
  });
  
  /// Establishes WebSocket connection
  Future<void> connect({Map<String, dynamic>? headers}) async {
    if (_isDisposed) throw StateError('Service is disposed');
    if (_state == ConnectionState.connecting || 
        _state == ConnectionState.connected) {
      return;  // Already connected or connecting
    }
    
    _updateState(ConnectionState.connecting);
    
    try {
      // Create WebSocket connection
      // IOWebSocketChannel for mobile/desktop, HtmlWebSocketChannel for web
      _channel = IOWebSocketChannel.connect(
        wsUrl,
        headers: headers,  // Auth tokens, etc.
        pingInterval: heartbeatInterval,  // Automatic ping from dart:io WebSocket
      );
      
      // Listen to incoming messages
      _subscription = _channel!.stream.listen(
        _onMessage,
        onError: _onError,
        onDone: _onDone,
        cancelOnError: false,  // Keep subscription alive on error
      );
      
      _updateState(ConnectionState.connected);
      _reconnectAttempts = 0;
      
      // Send any queued messages
      _flushMessageQueue();
      
      // Start custom heartbeat (if needed beyond pingInterval)
      _startHeartbeat();
      
      debugPrint('WebSocket connected to $wsUrl');
    } catch (e) {
      debugPrint('WebSocket connection error: $e');
      _updateState(ConnectionState.error);
      _scheduleReconnect();
    }
  }
  
  /// Sends message to server
  /// Queues message if offline
  void send(dynamic message) {
    final encoded = jsonEncode(message);
    
    if (isConnected && _channel != null) {
      _channel!.sink.add(encoded);
    } else {
      // Queue message for later
      _messageQueue.add(encoded);
      debugPrint('Message queued (offline)');
    }
  }
  
  /// Handles incoming messages
  void _onMessage(dynamic message) {
    try {
      final decoded = jsonDecode(message);
      _messageController.add(decoded);
    } catch (e) {
      // If not JSON, pass raw
      _messageController.add(message);
    }
  }
  
  /// Handles connection errors
  void _onError(error, StackTrace stackTrace) {
    debugPrint('WebSocket error: $error');
    _updateState(ConnectionState.error);
    // Don't reconnect immediately, let onDone handle it
  }
  
  /// Handles connection closure
  void _onDone() {
    debugPrint('WebSocket closed');
    _cleanupConnection();
    
    if (_shouldReconnect && !_isDisposed) {
      _scheduleReconnect();
    } else {
      _updateState(ConnectionState.disconnected);
    }
  }
  
  /// Schedules reconnection with exponential backoff
  void _scheduleReconnect() {
    if (_reconnectAttempts >= maxReconnectAttempts) {
      debugPrint('Max reconnection attempts reached');
      _updateState(ConnectionState.disconnected);
      return;
    }
    
    _reconnectAttempts++;
    _updateState(ConnectionState.reconnecting);
    
    // Exponential backoff: 1s, 2s, 4s, 8s... up to max 60s
    final delay = Duration(
      milliseconds: reconnectInterval.inMilliseconds * 
          (1 << (_reconnectAttempts - 1)),
    ).clamp(Duration(seconds: 1), Duration(seconds: 60));
    
    debugPrint('Reconnecting in ${delay.inSeconds}s (attempt $_reconnectAttempts)');
    
    _reconnectTimer?.cancel();
    _reconnectTimer = Timer(delay, () {
      if (!_isDisposed) connect();
    });
  }
  
  /// Sends queued messages once connected
  void _flushMessageQueue() {
    while (_messageQueue.isNotEmpty && isConnected) {
      final message = _messageQueue.removeAt(0);
      _channel?.sink.add(message);
    }
  }
  
  /// Heartbeat to detect half-open connections
  void _startHeartbeat() {
    _heartbeatTimer?.cancel();
    _heartbeatTimer = Timer.periodic(heartbeatInterval, (_) {
      if (isConnected) {
        // Send ping or custom heartbeat message
        send({'type': 'ping', 'timestamp': DateTime.now().toIso8601String()});
      }
    });
  }
  
  /// Updates connection state and notifies listeners
  void _updateState(ConnectionState newState) {
    if (_state != newState) {
      _state = newState;
      _stateController.add(newState);
    }
  }
  
  /// Cleans up connection resources
  void _cleanupConnection() {
    _subscription?.cancel();
    _subscription = null;
    _heartbeatTimer?.cancel();
    _heartbeatTimer = null;
    _channel = null;
  }
  
  /// Disconnects gracefully
  Future<void> disconnect() async {
    _shouldReconnect = false;
    _cleanupConnection();
    _updateState(ConnectionState.disconnected);
    debugPrint('WebSocket disconnected');
  }
  
  /// Disposes service permanently
  Future<void> dispose() async {
    _isDisposed = true;
    await disconnect();
    await _messageController.close();
    await _stateController.close();
    _reconnectTimer?.cancel();
  }
}

enum ConnectionState {
  disconnected,
  connecting,
  connected,
  reconnecting,
  error,
}

/// Typed message models for WebSocket communication
class WebSocketMessage {
  final String type;
  final dynamic payload;
  final String? id;
  final DateTime timestamp;
  
  WebSocketMessage({
    required this.type,
    this.payload,
    this.id,
    DateTime? timestamp,
  }) : timestamp = timestamp ?? DateTime.now();
  
  Map<String, dynamic> toJson() => {
        'type': type,
        'payload': payload,
        'id': id,
        'timestamp': timestamp.toIso8601String(),
      };
  
  factory WebSocketMessage.fromJson(Map<String, dynamic> json) {
    return WebSocketMessage(
      type: json['type'],
      payload: json['payload'],
      id: json['id'],
      timestamp: DateTime.parse(json['timestamp']),
    );
  }
}
```

**Explanation:**

- **`IOWebSocketChannel`**: Uses `dart:io` WebSocket implementation for native platforms (iOS/Android). For web, use `HtmlWebSocketChannel` which wraps browser WebSocket API.
- **`pingInterval`**: Dart's native WebSocket sends periodic ping frames to keep connection alive through NATs and firewalls. If no pong response received, connection is marked dead.
- **Exponential Backoff**: Reconnection delays double each time (1s, 2s, 4s...) up to 60s maximum. This prevents hammering a downed server with reconnection attempts while ensuring quick recovery from transient failures.
- **Message Queue**: While disconnected, messages are stored in `_messageQueue` and sent once connection restores. This enables "offline-first" messaging where users can compose messages without connectivity.
- **`cancelOnError: false`**: Ensures the stream subscription stays active even after errors, allowing us to handle errors gracefully rather than crashing the subscription.
- **Half-Open Connections**: TCP connections can appear alive while actually dead (e.g., when moving between cell towers). The heartbeat mechanism detects this by expecting periodic responses from the server.

---

## **21.4 Socket.IO Integration**

Socket.IO is a popular library that provides real-time bidirectional event-based communication with fallbacks for older browsers and automatic reconnection.

### **Socket.IO Client Setup**

```dart
// socket_io_service.dart
// Socket.IO client implementation for Flutter

import 'package:socket_io_client/socket_io_client.dart' as IO;
import 'package:flutter/foundation.dart';

/// Socket.IO service wrapper with Flutter-specific optimizations
/// Socket.IO provides:
/// - Automatic reconnection
/// - Binary support
/// - Multiplexing (namespaces)
/// - Fallback to HTTP long-polling if WebSocket fails
class SocketIOService {
  IO.Socket? _socket;
  final String serverUrl;
  final String? namespace;
  final Map<String, String>? authHeaders;
  
  // Event handlers storage
  final Map<String, List<Function>> _eventHandlers = {};
  
  bool _isConnected = false;
  bool _isConnecting = false;
  
  SocketIOService({
    required this.serverUrl,
    this.namespace,
    this.authHeaders,
  });
  
  /// Initialize and connect to Socket.IO server
  void connect() {
    if (_isConnecting || _isConnected) return;
    _isConnecting = true;
    
    // Build options
    final options = IO.OptionBuilder()
        .setTransports(['websocket', 'polling'])  // Fallback to polling
        .enableAutoConnect()                      // Connect immediately
        .enableReconnection()                     // Auto reconnect
        .setReconnectionAttempts(10)              // Max retries
        .setReconnectionDelay(1000)               // Initial delay 1s
        .setReconnectionDelayMax(5000)            // Max delay 5s
        .setRandomizationFactor(0.5)              // Randomize delay ±50%
        .setTimeout(5000)                         // Connection timeout
        .setExtraHeaders(authHeaders ?? {})       // Auth headers
        .build();
    
    // Create socket with optional namespace
    final url = namespace != null ? '$serverUrl/$namespace' : serverUrl;
    _socket = IO.io(url, options);
    
    // Connection events
    _socket!.onConnect((_) {
      _isConnected = true;
      _isConnecting = false;
      debugPrint('Socket.IO connected: ${_socket!.id}');
      _triggerEvent('connect', null);
    });
    
    _socket!.onDisconnect((reason) {
      _isConnected = false;
      _isConnecting = false;
      debugPrint('Socket.IO disconnected: $reason');
      _triggerEvent('disconnect', reason);
    });
    
    _socket!.onConnectError((error) {
      debugPrint('Socket.IO connection error: $error');
      _triggerEvent('error', error);
    });
    
    _socket!.onReconnect((attempt) {
      debugPrint('Socket.IO reconnected after $attempt attempts');
      _triggerEvent('reconnect', attempt);
    });
    
    _socket!.onReconnectAttempt((attempt) {
      debugPrint('Socket.IO reconnection attempt: $attempt');
    });
    
    _socket!.onReconnectError((error) {
      debugPrint('Socket.IO reconnection error: $error');
    });
    
    _socket!.onReconnectFailed((_) {
      debugPrint('Socket.IO reconnection failed');
      _triggerEvent('reconnect_failed', null);
    });
    
    // Error handling
    _socket!.onError((error) {
      debugPrint('Socket.IO error: $error');
      _triggerEvent('error', error);
    });
  }
  
  /// Register event listener
  /// [event]: Event name to listen for
  /// [handler]: Callback function(data)
  void on(String event, Function(dynamic) handler) {
    _eventHandlers.putIfAbsent(event, () => []).add(handler);
    
    _socket?.on(event, (data) {
      debugPrint('Received $event: $data');
      handler(data);
    });
  }
  
  /// Remove specific event listener
  void off(String event, [Function? handler]) {
    if (handler != null) {
      _eventHandlers[event]?.remove(handler);
    } else {
      _eventHandlers.remove(event);
    }
    _socket?.off(event);
  }
  
  /// Emit event to server
  /// [event]: Event name
  /// [data]: Payload (Map, List, String, etc.)
  /// [ack]: Optional callback for acknowledgment
  void emit(String event, dynamic data, [Function? ack]) {
    if (!_isConnected) {
      debugPrint('Warning: Emitting while disconnected');
      return;
    }
    
    if (ack != null) {
      _socket?.emitWithAck(event, data, ack: ack);
    } else {
      _socket?.emit(event, data);
    }
  }
  
  /// Join a room (server-side logic required)
  void joinRoom(String roomId) {
    emit('join', {'room': roomId});
  }
  
  /// Leave a room
  void leaveRoom(String roomId) {
    emit('leave', {'room': roomId});
  }
  
  /// Trigger registered local event handlers
  void _triggerEvent(String event, dynamic data) {
    final handlers = _eventHandlers[event] ?? [];
    for (final handler in handlers) {
      handler(data);
    }
  }
  
  /// Check connection status
  bool get isConnected => _isConnected;
  bool get isConnecting => _isConnecting;
  
  /// Get socket ID (available after connection)
  String? get socketId => _socket?.id;
  
  /// Manually disconnect
  void disconnect() {
    _socket?.disconnect();
    _isConnected = false;
    _isConnecting = false;
  }
  
  /// Manually reconnect
  void reconnect() {
    _socket?.connect();
  }
  
  /// Dispose and cleanup
  void dispose() {
    _eventHandlers.clear();
    _socket?.dispose();
    _socket = null;
  }
}

/// Example: Real-time collaboration using Socket.IO
class CollaborationManager {
  final SocketIOService _socket;
  final String documentId;
  
  CollaborationManager(this._socket, this.documentId);
  
  void initialize() {
    // Connect to collaboration namespace
    _socket.connect();
    
    // Listen for cursor movements from other users
    _socket.on('cursor_move', (data) {
      final userId = data['userId'];
      final position = data['position'];
      updateRemoteCursor(userId, position);
    });
    
    // Listen for document changes
    _socket.on('doc_change', (data) {
      final operation = data['operation'];  // OT or CRDT operation
      applyRemoteOperation(operation);
    });
    
    // Listen for user join/leave
    _socket.on('user_joined', (data) {
      showUserJoinedNotification(data['userName']);
    });
    
    // Join document room
    _socket.onConnect((_) {
      _socket.joinRoom('doc:$documentId');
    });
  }
  
  void sendCursorMove(int line, int column) {
    _socket.emit('cursor_move', {
      'documentId': documentId,
      'position': {'line': line, 'column': column},
    });
  }
  
  void sendTextChange(String operation) {
    _socket.emit('doc_change', {
      'documentId': documentId,
      'operation': operation,
      'timestamp': DateTime.now().millisecondsSinceEpoch,
    });
  }
  
  void updateRemoteCursor(String userId, Map position) {/* ... */}
  void applyRemoteOperation(String operation) {/* ... */}
  void showUserJoinedNotification(String userName) {/* ... */}
}
```

**Explanation:**

- **Transports**: `['websocket', 'polling']` specifies transport preference. Socket.IO first tries WebSocket, but if it fails (corporate proxies, old browsers), it falls back to HTTP long-polling.
- **Reconnection Strategy**: Exponential backoff with randomization (`setRandomizationFactor`) prevents thundering herd problems when a server comes back online (all clients don't reconnect simultaneously).
- **Acknowledgments**: `emitWithAck` expects a response from the server. If the server calls the callback, the client receives confirmation that the message was processed.
- **Namespaces**: Socket.IO supports multiplexing multiple logical connections over one physical connection via namespaces (`/chat`, `/notifications`). This saves resources compared to multiple WebSocket connections.
- **Rooms**: A server-side concept where sockets can join "rooms" (channels) to receive targeted broadcasts. The client emits `join` events, and the server manages room membership.

---

## **21.5 Server-Sent Events (SSE)**

Server-Sent Events provide a one-way stream from server to client over HTTP. Simpler than WebSockets for use cases where only the server needs to push data (news feeds, stock tickers, logs).

### **SSE Implementation**

```dart
// sse_service.dart
// Server-Sent Events client implementation
// SSE uses standard HTTP with text/event-stream MIME type

import 'dart:async';
import 'dart:convert';
import 'dart:io';
import 'package:flutter/foundation.dart';
import 'package:http/http.dart' as http;

/// Server-Sent Events (SSE) client
/// SSE characteristics:
/// - One-way: Server to client only
/// - HTTP-based: Works through corporate proxies/firewalls
/// - Automatic reconnection: Built into browser spec (we implement manually)
/// - Text only: Binary data must be base64 encoded
class SSEService {
  final String url;
  final Map<String, String>? headers;
  final Duration retryInterval;
  
  http.Client? _client;
  StreamSubscription? _subscription;
  Timer? _retryTimer;
  
  final _eventController = StreamController<SSEEvent>.broadcast();
  final _connectionController = StreamController<bool>.broadcast();
  
  bool _isConnected = false;
  bool _shouldReconnect = true;
  String? _lastEventId;  // For resuming stream
  
  Stream<SSEEvent> get eventStream => _eventController.stream;
  Stream<bool> get connectionStream => _connectionController.stream;
  bool get isConnected => _isConnected;
  
  SSEService({
    required this.url,
    this.headers,
    this.retryInterval = const Duration(seconds: 3),
  });
  
  /// Connect to SSE endpoint
  void connect() {
    if (_isConnected) return;
    
    _client?.close();
    _client = http.Client();
    
    final request = http.Request('GET', Uri.parse(url));
    request.headers.addAll({
      'Accept': 'text/event-stream',
      'Cache-Control': 'no-cache',
      if (_lastEventId != null) 'Last-Event-ID': _lastEventId!,
      ...?headers,
    });
    
    _client!.send(request).then((response) {
      if (response.statusCode == 200) {
        _handleResponse(response);
      } else {
        _handleError('HTTP ${response.statusCode}');
      }
    }).catchError((error) {
      _handleError(error.toString());
    });
  }
  
  /// Process streaming response
  void _handleResponse(http.StreamedResponse response) {
    _setConnectionState(true);
    
    // SSE messages are separated by double newlines
    // Format:
    // id: 123\n
    // event: message\n
    // data: {"key": "value"}\n\n
    
    String buffer = '';
    
    _subscription = response.stream
        .transform(utf8.decoder)
        .listen(
          (data) {
            buffer += data;
            
            // Process complete events (separated by \n\n)
            while (buffer.contains('\n\n')) {
              final index = buffer.indexOf('\n\n');
              final eventData = buffer.substring(0, index);
              buffer = buffer.substring(index + 2);
              
              _processEvent(eventData);
            }
          },
          onError: (error) {
            _handleError('Stream error: $error');
          },
          onDone: () {
            _setConnectionState(false);
            if (_shouldReconnect) _scheduleReconnect();
          },
        );
  }
  
  /// Parse SSE event format
  void _processEvent(String eventData) {
    final lines = eventData.split('\n');
    String? id;
    String? event;
    StringBuffer dataBuffer = StringBuffer();
    
    for (final line in lines) {
      if (line.startsWith('id:')) {
        id = line.substring(3).trim();
      } else if (line.startsWith('event:')) {
        event = line.substring(6).trim();
      } else if (line.startsWith('data:')) {
        if (dataBuffer.isNotEmpty) dataBuffer.write('\n');
        dataBuffer.write(line.substring(5).trim());
      } else if (line.startsWith('retry:')) {
        // Server suggested retry interval
        final retryMs = int.tryParse(line.substring(6).trim());
        if (retryMs != null) {
          // Update retry interval
        }
      }
    }
    
    if (id != null) _lastEventId = id;
    
    final eventObj = SSEEvent(
      id: id,
      event: event ?? 'message',
      data: dataBuffer.toString(),
    );
    
    _eventController.add(eventObj);
  }
  
  /// Handle connection errors
  void _handleError(String error) {
    debugPrint('SSE Error: $error');
    _setConnectionState(false);
    if (_shouldReconnect) _scheduleReconnect();
  }
  
  /// Schedule reconnection
  void _scheduleReconnect() {
    _retryTimer?.cancel();
    _retryTimer = Timer(retryInterval, () {
      if (_shouldReconnect) connect();
    });
  }
  
  /// Update connection state
  void _setConnectionState(bool connected) {
    if (_isConnected != connected) {
      _isConnected = connected;
      _connectionController.add(connected);
    }
  }
  
  /// Disconnect gracefully
  void disconnect() {
    _shouldReconnect = false;
    _setConnectionState(false);
    _subscription?.cancel();
    _retryTimer?.cancel();
    _client?.close();
    _client = null;
  }
  
  /// Dispose resources
  void dispose() {
    disconnect();
    _eventController.close();
    _connectionController.close();
  }
}

/// SSE Event model
class SSEEvent {
  final String? id;
  final String event;
  final String data;
  
  SSEEvent({
    this.id,
    required this.event,
    required this.data,
  });
  
  /// Parse data as JSON
  dynamic get jsonData {
    try {
      return jsonDecode(data);
    } catch (e) {
      return null;
    }
  }
  
  @override
  String toString() => 'SSEEvent(id: $id, event: $event, data: $data)';
}

/// Example: Live news feed using SSE
class NewsFeedWidget extends StatefulWidget {
  @override
  _NewsFeedWidgetState createState() => _NewsFeedWidgetState();
}

class _NewsFeedWidgetState extends State<NewsFeedWidget> {
  final _sseService = SSEService(
    url: 'https://api.example.com/news/stream',
    headers: {'Authorization': 'Bearer token'},
  );
  
  final List<NewsItem> _news = [];
  
  @override
  void initState() {
    super.initState();
    
    _sseService.eventStream.listen((event) {
      if (event.event == 'news') {
        final newsData = event.jsonData;
        if (newsData != null) {
          setState(() {
            _news.insert(0, NewsItem.fromJson(newsData));
          });
        }
      } else if (event.event == 'ping') {
        // Keep-alive, ignore
      }
    });
    
    _sseService.connect();
  }
  
  @override
  void dispose() {
    _sseService.dispose();
    super.dispose();
  }
  
  @override
  Widget build(BuildContext context) {
    return ListView.builder(
      itemCount: _news.length,
      itemBuilder: (context, index) {
        final item = _news[index];
        return ListTile(
          title: Text(item.title),
          subtitle: Text(item.summary),
          trailing: Text(
            timeAgo(item.timestamp),
            style: TextStyle(fontSize: 12),
          ),
        );
      },
    );
  }
}

class NewsItem {
  final String title;
  final String summary;
  final DateTime timestamp;
  
  NewsItem({required this.title, required this.summary, required this.timestamp});
  
  factory NewsItem.fromJson(Map<String, dynamic> json) {
    return NewsItem(
      title: json['title'],
      summary: json['summary'],
      timestamp: DateTime.parse(json['timestamp']),
    );
  }
}

String timeAgo(DateTime date) {
  final diff = DateTime.now().difference(date);
  if (diff.inMinutes < 60) return '${diff.inMinutes}m ago';
  if (diff.inHours < 24) return '${diff.inHours}h ago';
  return '${diff.inDays}d ago';
}
```

**Explanation:**

- **SSE Format**: Messages follow the format `field: value\n` with fields: `id` (for resuming), `event` (event type), `data` (payload), `retry` (suggested reconnection time). Messages end with double newline (`\n\n`).
- **`Last-Event-ID`**: When reconnecting, the client sends the ID of the last received event. The server can then replay missed events, ensuring no data loss during brief disconnections.
- **Text-only**: SSE only supports UTF-8 text. Binary data must be base64 encoded, making it less efficient than WebSockets for binary protocols.
- **HTTP Advantages**: Works through corporate proxies that block WebSocket upgrades, supports HTTP/2 multiplexing, and can leverage standard HTTP authentication/authorization mechanisms.
- **Unidirectional**: SSE is perfect for server-to-client streaming (stock prices, social media feeds, logs) where the client doesn't need to send data back frequently.

---

## **Chapter Summary**

In this chapter, we explored real-time communication patterns in Flutter beyond traditional REST APIs:

### **Key Takeaways:**

1. **GraphQL Setup**: Use `graphql_flutter` with link composition (Auth -> Error -> HTTP). Configure `HiveStore` for offline persistence and normalize cache using `dataIdFromObject` with `__typename:id` pattern.

2. **GraphQL Operations**: Queries support caching policies (`cacheAndNetwork`, `networkOnly`). Mutations support optimistic updates for instant UI feedback and `update` callbacks to modify cache after success. Subscriptions use WebSocket transport for server-pushed updates.

3. **WebSocket Management**: Implement exponential backoff for reconnection (1s, 2s, 4s...), heartbeat/ping-pong to detect dead connections, and message queuing for offline scenarios. Use `IOWebSocketChannel` for native, `HtmlWebSocketChannel` for web.

4. **Socket.IO**: Provides fallbacks (WebSocket -> HTTP polling), automatic reconnection with jitter, acknowledgments for message delivery confirmation, and namespaces for connection multiplexing. Ideal for chat, gaming, and collaboration features.

5. **Server-Sent Events**: HTTP-based unidirectional streaming perfect for news feeds, stock tickers, and logs. Supports automatic reconnection with `Last-Event-ID` for event replay. Works through restrictive proxies but limited to text data.

### **When to Choose Which:**

- **GraphQL**: When you need strongly-typed APIs, efficient data fetching (no over-fetching), and real-time subscriptions in the same schema.
- **Raw WebSocket**: When you need maximum control, binary data support, or custom protocols (gaming, low-latency trading).
- **Socket.IO**: When you need reliability (fallbacks), room-based broadcasting, or broad browser compatibility.
- **SSE**: When you only need server-to-client streaming, want HTTP compatibility (proxies, auth), or have simple push notification needs.

### **Next Steps:**

The next chapter (Chapter 22) will cover **Local Data Persistence**, including:
- SharedPreferences for simple key-value storage
- SQLite with sqflite for relational data
- NoSQL options (Hive, ObjectBox) for high-performance document storage
- File system operations and path management
- Data synchronization strategies (offline-first architecture)

---

**End of Chapter 21**

<div style='width:100%; display:flex; justify-content:space-between; align-items:center; margin: 1em 0;'>
  <a href='20. authentication_and_security.ipynb' style='font-weight:bold; font-size:1.05em;'>&larr; Previous</a>
  <a href='../TOC.md' style='font-weight:bold; font-size:1.05em; text-align:center;'>Table of Contents</a>
  <a href='22. local_data_persistence.ipynb' style='font-weight:bold; font-size:1.05em;'>Next &rarr;</a>
</div>
